<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->

**CURSO DE PÓS-GRADUAÇÃO EM CIÊNCIA DE DADOS (BIG DATA PROCESSING AND ANALYTICS)**<p>
**Componente curricular:** MINERAÇÃO E ANÁLISE DE DADOS [TURMA 01D] - 2023/1 - Trilha 4.<br>

**Aluno:** ROBSON DE FREITAS SAMPAIO.<br>

**URL deste notebook:** https://github.com/rfsampaio/postgraduate_data_science/blob/main/notebooks/MA_T4_A.ipynb<br>
***

## Atividade - Trilha 4 - Parte A - Classificação com "scikit-learn"

#### CASO: Controle de Qualidade de Peças de uma Indústria.

A partir deste notebook você vai criar e avaliar diferentes modelos de classificação para prever a qualidade de peças de uma indústria. 

Em seguida, com base nos seus resultados, responda o **questionário do Moodle**. (**dica**: o questionário é de alternativas e as respostas irão ajudá-lo a verificar os seus resultados ao longo do programa). 

### Passos:

#### Importando as bibliotecas básicas.

In [129]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


#### Exploração dos dados.

`pieces` é um data set para o controle de qualidade de peças produzidas por uma indústria. São empregadas quatro medidas (A, B, C e D) para o controle da qualidade das peças. As peças são então *Accept, Refurbish* ou *Reject* segundo o controle de qualidade (atributo `Quality`). A indústria ainda conta com 3 unidades para a produção dessas peças (atributo `Unit`). 

**Objetivo:** Seu objetivo é criar modelos supervisionados de classificação para predição da qualidade das peças e empregar o melhor modelo obtido (maior acuracidade) na predição da qualidade de novos casos.

In [130]:
df = pd.read_csv('../data/pieces1.csv')
df.head()


,id,A,B,C,D,Quality,Unit
0,559,4.9,3.1,1.5,0.1,Reject,SP
1,629,4.8,3.4,1.6,0.2,Reject,SP
2,192,6.7,2.5,5.8,1.8,Refurbish,RJ
3,359,7.6,3.0,6.6,2.1,Refurbish,RJ
4,9,4.9,3.1,1.5,0.1,Reject,RJ


#### Inspecione os dados.

In [131]:
display(df.dtypes)
display(df.shape)

id           int64
A          float64
B          float64
C          float64
D          float64
Quality     object
Unit        object
dtype: object

(500, 7)

#### Preparação dos dados.

Aqui você vai preparar o dado para o uso nos modelos. Você fará a seleção de atributos, tratamento de valores nulos, o encode dos dados, normalização e a separação dos dados de treinamento e teste.

#### Seleção de Atributos.

Elimine atributos que não fazem parte do treinamento.

In [132]:
df = df.drop(columns=['id'])
df.head()

,A,B,C,D,Quality,Unit
0,4.9,3.1,1.5,0.1,Reject,SP
1,4.8,3.4,1.6,0.2,Reject,SP
2,6.7,2.5,5.8,1.8,Refurbish,RJ
3,7.6,3.0,6.6,2.1,Refurbish,RJ
4,4.9,3.1,1.5,0.1,Reject,RJ


#### Tratamento de Dados Ausentes.

Verifique as presença de valores ausentes e, se houverem, substitua pelo valor médio dos atributos.

In [133]:
# finding NaNs
df.isnull().sum() / len(df)

A          0.000
B          0.018
C          0.012
D          0.008
Quality    0.000
Unit       0.000
dtype: float64

In [134]:
# filling NaNs with mean
df['B'] = df['B'].fillna(df['B'].mean())
df['C'] = df['C'].fillna(df['C'].mean())
df['D'] = df['D'].fillna(df['D'].mean())

display(df.isnull().sum() / len(df))
display(df.shape)



A          0.0
B          0.0
C          0.0
D          0.0
Quality    0.0
Unit       0.0
dtype: float64

(500, 6)

#### Hot encode.

Faça os *Hot encodes* **necessários** dos dados. Lembre-se de verificar se de fato todos os dados categóricos precisam dessa transformação.

In [135]:
from sklearn.preprocessing import OneHotEncoder

# split the dataframe into its not encoded and encoded features
df2 = df[['A', 'B', 'C', 'D', 'Quality']]
df2_enc = df[['Unit']]

# create a OneHotEncoder and encode the categorical features
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
df2_encoded = encoder.fit_transform(df2_enc)

# create the names for the encoded categorical features
categorical_columns = [f'{col}_{cat}' for i, col in enumerate(df2_enc.columns)
                       for cat in encoder.categories_[i]]

# put the encoded categorical features into a dataframe and join with the not encoded features
one_hot_features = pd.DataFrame(df2_encoded, columns=categorical_columns)
df = df2.join(one_hot_features)

display(df.head())
display(df.shape)

,A,B,C,D,Quality,Unit_BH,Unit_RJ,Unit_SP
0,4.9,3.1,1.5,0.1,Reject,0.0,0.0,1.0
1,4.8,3.4,1.6,0.2,Reject,0.0,0.0,1.0
2,6.7,2.5,5.8,1.8,Refurbish,0.0,1.0,0.0
3,7.6,3.0,6.6,2.1,Refurbish,0.0,1.0,0.0
4,4.9,3.1,1.5,0.1,Reject,0.0,1.0,0.0


(500, 8)

#### Normalize os dados.

Normalize os dados com o `StandardScaler` (melhor) ou a função `scale`.

$$ z_i = \frac{x_i - \bar{x}}{\sigma(x)}$$

E não esqueça de excluir dados não úteis ao treinamento. Depois de normalizar os dados os valores devem apresentar média próxima de 0 e desvio padrão próximo de 1.

In [136]:
from sklearn.preprocessing import StandardScaler

# define standard scaler
scaler = StandardScaler()

# transform data
df_scaled = pd.DataFrame(scaler.fit_transform(df.drop(columns=['Quality'])),
                         columns=['A', 'B', 'C', 'D', 'Unit_BH', 'Unit_RJ', 'Unit_SP'])
df_scaled = pd.concat([df_scaled, df[['Quality']]], axis=1)

display(df_scaled)

,A,B,C,D,Unit_BH,Unit_RJ,Unit_SP,Quality
0,-1.138645,0.053976,-1.316365,-1.476929,-0.657773,-0.577350,1.110019,Reject
1,-1.261132,0.751404,-1.258818,-1.346836,-0.657773,-0.577350,1.110019,Reject
2,1.066132,-1.340880,1.158169,0.734661,-0.657773,1.732051,-0.900885,Refurbish
3,2.168521,-0.178500,1.618547,1.124942,-0.657773,1.732051,-0.900885,Refurbish
4,-1.138645,0.053976,-1.316365,-1.476929,-0.657773,1.732051,-0.900885,Reject
...,...,...,...,...,...,...,...,...
495,-1.506108,1.216356,-1.604102,-1.346836,-0.657773,-0.577350,1.110019,Reject
496,1.311107,0.286452,1.100621,1.385129,-0.657773,1.732051,-0.900885,Refurbish
497,1.066132,0.053976,1.043074,1.515223,-0.657773,1.732051,-0.900885,Refurbish
498,-0.281232,-0.178500,0.179864,0.084194,-0.657773,-0.577350,1.110019,Accept


(Moodle) **Q1**. Após todas as transformações a soma dos valores absolutos ( `np.abs()` ) dos atributos referentes ao hot encode dos *BH*, *RJ* e *SP* são respectivamente?

*Empregue esta pergunta como ponto de checagem das suas transformações. Se não encontrar os seus valores dentre as alternativas há provavelmente um erro e você deve revisar as transformações.*

In [137]:
print(np.abs(df_scaled['Unit_BH']).sum())
print(np.abs(df_scaled['Unit_RJ']).sum())
print(np.abs(df_scaled['Unit_SP']).sum())

459.12525524087647
433.0127018922194
497.2886485734416


#### Conjuntos de Treinamento e Teste.

Separe os dados de treinamento e teste (30%, estratificados pela variável objetivo e com random_state = 1984). Não empregue outros parâmetros não solicitados.

In [138]:
from sklearn.model_selection import train_test_split

# define X and y
X = df_scaled.drop(columns=['Quality'])
y = df_scaled['Quality']

# split train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=1984)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(350, 7) (150, 7) (350,) (150,)


#### **K-Vizinhos mais Próximos.**

Empregue o `GridSearchCV` com 5 partições e `scoring='accuracy'` para encontrar o melhor modelo entre os valores de $k$ de $4$ a $11$ e empregando as métricas euclidiana e manhattan. Verifique o modelo obtido e o `classification_report` desse modelo.

Você pode querer empregar o modelo de código no final desse notebook que faz a mesma construção para Árvores de Decisão.

In [139]:
from sklearn.neighbors import KNeighborsClassifier
KNeighborsClassifier().get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [140]:
from sklearn import neighbors
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

base_estimator = neighbors.KNeighborsClassifier()

# k values from 4 to 11 and metrics = 'euclidean', 'manhattan'
param_grid = {'n_neighbors': [4,5,6,7,8,9,10,11], 'metric': ['euclidean','manhattan']}

# GridSearchCV with cv=5 and scoring='accuracy'
clf = GridSearchCV(base_estimator, param_grid, cv=5, scoring='accuracy')

# fit the model
clf.fit(X_train, y_train)

# print(clf.cv_results_)
print(clf.best_estimator_)

print()
print('Detailed classification report:')
print()
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print()

KNeighborsClassifier(metric='manhattan', n_neighbors=7)

Detailed classification report:

              precision    recall  f1-score   support

      Accept       0.88      0.98      0.93        47
   Refurbish       0.98      0.89      0.93        55
      Reject       1.00      1.00      1.00        48

    accuracy                           0.95       150
   macro avg       0.95      0.96      0.95       150
weighted avg       0.96      0.95      0.95       150




(Moodle) **Q2**. Qual o melhor modelo de Knn obtido e qual a sua acuracidade? 

(Moodle) **Q3**. Qual classe teve os elementos mais erroneamente classificados? 

#### **Regressão Logística.**

Empregue o `GridSearchCV` com 5 partições e `scoring='accuracy'` para avaliar o modelo de regressão logística. Empregue somente os parâmetros padrão do `sciki-learn` para regressão logística, para isso basta empregar `param_grid = {}`. Verifique os resultados obtidos desse modelo com o `classification_report`. 

Você pode querer empregar o modelo de código no final desse notebook que faz a mesma construção para Árvores de Decisão.

In [141]:
from sklearn.linear_model import LogisticRegression
LogisticRegression().get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [142]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

base_estimator = LogisticRegression()

# default params for Logistic Regression
param_grid = {}

# GridSearchCV with cv=5 and scoring='accuracy'
clf = GridSearchCV(base_estimator, param_grid, cv=5, scoring='accuracy')

# fit the model
clf.fit(X_train, y_train)

# print(clf.cv_results_)
print(clf.best_estimator_)

print()
print('Detailed classification report:')
print()
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print()


LogisticRegression()

Detailed classification report:

              precision    recall  f1-score   support

      Accept       0.92      1.00      0.96        47
   Refurbish       1.00      0.93      0.96        55
      Reject       1.00      1.00      1.00        48

    accuracy                           0.97       150
   macro avg       0.97      0.98      0.97       150
weighted avg       0.98      0.97      0.97       150




(Moodle) **Q4**. Qual a acuracidade do modelo de regressão logística e qual classe teve mais falsos positivos? 

#### **Aplicando o melhor Modelo.**

Empregue o melhor modelo obtido acima para estimar a qualidade das 5 peças do conjunto abaixo.

In [143]:
df_new = pd.read_csv('../data/pieces_new.csv')
display(df_new.head())
display(df_new.dtypes)
display(df_new.shape)

,id,A,B,C,D,Unit
0,182,5.5,2.6,4.4,1.2,SP
1,345,5.8,2.7,4.1,1.0,SP
2,42,5.1,3.5,1.4,0.2,SP
3,37,6.3,2.5,4.9,1.5,BH
4,61,6.0,3.0,4.8,1.8,RJ


id        int64
A       float64
B       float64
C       float64
D       float64
Unit     object
dtype: object

(5, 6)

#### Preparação dos Dados.

Lembre-se, as mesmas operações feitas com os dados de treinamento precisarão ser feitas aqui também.

#### Seleção de Atributos. 

In [144]:
df_new = df_new.drop(columns=['id'])
df_new.head()

,A,B,C,D,Unit
0,5.5,2.6,4.4,1.2,SP
1,5.8,2.7,4.1,1.0,SP
2,5.1,3.5,1.4,0.2,SP
3,6.3,2.5,4.9,1.5,BH
4,6.0,3.0,4.8,1.8,RJ


In [145]:
# finding NaNs
df_new.isnull().sum() / len(df_new)

A       0.0
B       0.0
C       0.0
D       0.0
Unit    0.0
dtype: float64

#### Hot encode.

In [146]:
# split the dataframe into its not encoded and encoded features
df2 = df_new[['A', 'B', 'C', 'D']]
df2_enc = df_new[['Unit']]

# create a OneHotEncoder and encode the categorical features
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
df2_encoded = encoder.fit_transform(df2_enc)

# create the names for the encoded categorical features
categorical_columns = [f'{col}_{cat}' for i, col in enumerate(df2_enc.columns)
                       for cat in encoder.categories_[i]]

# put the encoded categorical features into a dataframe and join with the not encoded features
one_hot_features = pd.DataFrame(df2_encoded, columns=categorical_columns)
df_new = df2.join(one_hot_features)

display(df_new.head())
display(df_new.shape)

,A,B,C,D,Unit_BH,Unit_RJ,Unit_SP
0,5.5,2.6,4.4,1.2,0.0,0.0,1.0
1,5.8,2.7,4.1,1.0,0.0,0.0,1.0
2,5.1,3.5,1.4,0.2,0.0,0.0,1.0
3,6.3,2.5,4.9,1.5,1.0,0.0,0.0
4,6.0,3.0,4.8,1.8,0.0,1.0,0.0


(5, 7)

#### Normalize os dados.

In [147]:
# define standard scaler
scaler = StandardScaler()

# transform data
df_scaled_new = pd.DataFrame(scaler.fit_transform(df_new),
                             columns=['A', 'B', 'C', 'D', 'Unit_BH', 'Unit_RJ', 'Unit_SP'])

display(df_scaled_new)

,A,B,C,D,Unit_BH,Unit_RJ,Unit_SP
0,-0.581402,-0.720003,0.371480,0.110581,-0.5,-0.5,0.816497
1,0.145350,-0.443079,0.139305,-0.258023,-0.5,-0.5,0.816497
2,-1.550405,1.772316,-1.950268,-1.732443,-0.5,-0.5,0.816497
3,1.356604,-0.996928,0.758438,0.663489,2.0,-0.5,-1.224745
4,0.629852,0.387694,0.681046,1.216396,-0.5,2.0,-1.224745


#### Predição.

In [148]:
# Logistic Regression was the best model
base_estimator = LogisticRegression()

# default params for Logistic Regression
param_grid = {}

# GridSearchCV with cv=5 and scoring='accuracy'
clf = GridSearchCV(base_estimator, param_grid, cv=5, scoring='accuracy')

# fit the model
clf.fit(X_train, y_train)

# executing the predictions
df_scaled_new['Prediction'] = clf.predict(df_scaled_new)

# showing predictions
display(df_scaled_new)

# showing the model score
display(f"Model's Score: {clf.score(X,y)}")

,A,B,C,D,Unit_BH,Unit_RJ,Unit_SP,Prediction
0,-0.581402,-0.720003,0.371480,0.110581,-0.5,-0.5,0.816497,Accept
1,0.145350,-0.443079,0.139305,-0.258023,-0.5,-0.5,0.816497,Accept
2,-1.550405,1.772316,-1.950268,-1.732443,-0.5,-0.5,0.816497,Reject
3,1.356604,-0.996928,0.758438,0.663489,2.0,-0.5,-1.224745,Refurbish
4,0.629852,0.387694,0.681046,1.216396,-0.5,2.0,-1.224745,Refurbish


"Model's Score: 0.978"

(Moodle) **Q5**. Quais as classes obtidas com o melhor modelo para os cinco novos casos?

#### **Apêndice: Árvore de Decisão (RESOLVIDO).**

Este é apenas um exemplo empregando o `GridSearchCV` com 5 partições e `scoring='accuracy'` para avaliar um modelo de Árvore de Decisão com diferentes parâmetros. Você irá estudar mais detalhes do modelo de Árvore de Decisão na próxima trilha. Mas ele é um classificador, assim como o modelo de K-Vizinhos mais Próximos e a Regressão Logística, e o uso desse estimador segue os mesmos moldes desses estimadores. Você pode assim aplicá-lo do mesmo modo.

In [149]:
from sklearn.tree import DecisionTreeClassifier
DecisionTreeClassifier().get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [150]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

base_estimator = DecisionTreeClassifier()

param_grid = {'criterion': ['gini', 'entropy'], 'max_depth': [ None, 3 ]}

clf = GridSearchCV(base_estimator, param_grid, cv=5, scoring='accuracy')
clf.fit(X_train, y_train)

# print(clf.cv_results_)
print(clf.best_estimator_)

print()
print('Detailed classification report:')
print()
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print()

DecisionTreeClassifier(criterion='entropy')

Detailed classification report:

              precision    recall  f1-score   support

      Accept       1.00      1.00      1.00        47
   Refurbish       1.00      1.00      1.00        55
      Reject       1.00      1.00      1.00        48

    accuracy                           1.00       150
   macro avg       1.00      1.00      1.00       150
weighted avg       1.00      1.00      1.00       150


